In [3]:
import pickle,os,sys,gc
import pandas as pd
import numpy as np
import ssdeep
import glob
from tqdm import tqdm
import functools,operator
from datetime import datetime
from sklearn.metrics import *
from sklearn.metrics.pairwise import *
from collections import Counter
# import matplotlib相關套件
import matplotlib.pyplot as plt
# import字型管理套件
from matplotlib.font_manager import FontProperties
from matplotlib import pyplot
from operator import itemgetter
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

In [4]:
#  import sys,os
print(sys.executable)
print(sys.version)
print(sys.version_info)
import findspark
# os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3' #'/home/antslab/anaconda3/bin/python'
findspark.init()
import pyspark
# from pyspark import SparkContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import Row, SQLContext, SparkSession, window
from pyspark import SparkConf, SparkContext
# Import data types
from pyspark.sql.types import *
import  pyspark.sql.functions as F
conf = SparkConf()
# conf.set("spark.local.dir", "/mnt/ssd240g/data/Leo_Spark_Home/tmp")
conf.set("spark.executor.cores","12")
conf.setMaster("local[12]")
conf.set("spark.driver.memory","265g") #滿載165G/
conf.set("spark.executor.memory", "110g")
conf.set("spark.memory.offHeap.enabled","true")
conf.set("spark.memory.offHeap.size","105g")
conf.set("spark.driver.extraJavaOptions","-Xss95g")
conf.set("spark.driver.extraJavaOptions","-Xms95g")
# conf.set("spark.memory.storageFraction","0.2")
# conf.set("spark.driver.maxResultSize","0")
# conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
# conf.set("spark.sql.execution.arrow.enabled", "true")
conf.set("spark.kubernetes.pyspark.pythonVersion","3")
conf.set("spark.network.timeout","1500s")
conf.set("spark.sql.shuffle.partitions",140000)
conf.set("spark.driver.maxResultSize", "170g")

# spark = SparkSession.builder.appName("pcap_analyzer").config(conf=conf).getOrCreate() #pcap_analyzer pcapAnalyzer
spark = SparkSession.builder.master("local").appName("Leo_spark_123").config(conf=conf).getOrCreate()
# spark.conf.set("spark.sql.execution.arrow.enabled", True)
spark.conf.set("spark.sql.debug.maxToStringFields", 300000)
spark.conf.set("spark.sql.hive.filesourcePartitionFileCacheSize",120*1024*1024*1024)

import databricks.koalas as ks
ks.options.display.max_rows = 20
ks.set_option('compute.max_rows', None)
# ks.set_option('compute.ops_on_diff_frames', True)
ks.set_option('compute.default_index_type', 'distributed')
print(ks.__version__,spark.version)
spark

/home/antslab/anaconda3/bin/python
3.7.10 (default, Feb 26 2021, 18:47:35) 
[GCC 7.3.0]
sys.version_info(major=3, minor=7, micro=10, releaselevel='final', serial=0)
1.7.0 3.0.0


In [8]:
date_li = ['20200106','20200107','20200108','20200109','20200110',
           '20200111','20200112'] #改
# date_li.reverse()
isp_li = ['中嘉寬頻','亞太電信','台固媒體','台灣固網','台灣大哥大','中華電信','凱擘','台灣之星',
        '台灣基礎開發','台灣碩網']
# protocols_need = ['http','mysql','ftp','smb','smtp','imap','pop','rpc','ssh','telnet','sip']

# isp_li.reverse()
file_name = "_".join(sorted(date_li)) #date_li 改!!
min_date = str(min(date_li))
max_date = str(max(date_li))
country_df = pd.DataFrame() #1st time

In [11]:
for time in tqdm(date_li):
    for isp in isp_li:
#         if time+'_'+isp in processed_li_rev:
#             continue #之前有一部分已經跑過了        
        print("=== Now is about to run:",time,isp,"===")
        in_file = 'hdfs://192.168.50.123:8020/user/hdfs/parquet/'+str(time[:4])+'_'+str(time[4:6])+'_'+str(time[6:])+'/'+isp+'/session_parquet/'+str(time)+'_session.parquet'
#         in_file = 'hdfs://192.168.50.200/user/hdfs/parquet/'+str(time[:4])+'_'+str(time[4:6])+'_'+str(time[6:])+'/'+isp+'/session_parquet/'+str(time)+'_session.parquet'
        kdf = ks.read_parquet(in_file)
#         kdf_nopayload = kdf[(kdf.tcp_i_payload_list.astype(str)=="[]")|(kdf.tcp_o_payload_list.astype(str)=="[]")]
        kdf = kdf[['country','tcp_dstport','ip_dst','session_i_tt_packet','frame_i_max_protocols']]
        gc.collect()
        kdf = kdf.to_pandas()
        kdf_gby_co = kdf.groupby('country').agg(list)
        kdf_gby_co = kdf_gby_co[kdf_gby_co.index.isin(['China','Taiwan','Ireland'])]
        t_index = kdf_gby_co.index.tolist()
        t_index = [time+'_'+isp+'_'+str(ind) for ind in t_index]
        kdf_gby_co.index = t_index        
        country_df = country_df.append(kdf_gby_co)
        gc.collect()
        pickle.dump(obj=country_df,file=open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/tri_country_all_dist.pkl','wb'),protocol=pickle.HIGHEST_PROTOCOL)
country_df        

  0%|          | 0/7 [00:00<?, ?it/s]

=== Now is about to run: 20200106 中嘉寬頻 ===
=== Now is about to run: 20200106 亞太電信 ===
=== Now is about to run: 20200106 台固媒體 ===
=== Now is about to run: 20200106 台灣固網 ===
=== Now is about to run: 20200106 台灣大哥大 ===
=== Now is about to run: 20200106 中華電信 ===
=== Now is about to run: 20200106 凱擘 ===
=== Now is about to run: 20200106 台灣之星 ===
=== Now is about to run: 20200106 台灣基礎開發 ===
=== Now is about to run: 20200106 台灣碩網 ===


 14%|█▍        | 1/7 [2:39:14<15:55:29, 9554.92s/it]

=== Now is about to run: 20200107 中嘉寬頻 ===
=== Now is about to run: 20200107 亞太電信 ===
=== Now is about to run: 20200107 台固媒體 ===
=== Now is about to run: 20200107 台灣固網 ===
=== Now is about to run: 20200107 台灣大哥大 ===
=== Now is about to run: 20200107 中華電信 ===
=== Now is about to run: 20200107 凱擘 ===
=== Now is about to run: 20200107 台灣之星 ===
=== Now is about to run: 20200107 台灣基礎開發 ===
=== Now is about to run: 20200107 台灣碩網 ===


 29%|██▊       | 2/7 [6:00:04<15:18:30, 11022.16s/it]

...

=== Now is about to run: 20200111 中嘉寬頻 ===
=== Now is about to run: 20200111 亞太電信 ===
=== Now is about to run: 20200111 台固媒體 ===
=== Now is about to run: 20200111 台灣固網 ===
=== Now is about to run: 20200111 台灣大哥大 ===
=== Now is about to run: 20200111 中華電信 ===
=== Now is about to run: 20200111 凱擘 ===
=== Now is about to run: 20200111 台灣之星 ===
=== Now is about to run: 20200111 台灣基礎開發 ===
=== Now is about to run: 20200111 台灣碩網 ===


 86%|████████▌ | 6/7 [19:36:55<3:31:48, 12708.93s/it]

=== Now is about to run: 20200112 中嘉寬頻 ===
=== Now is about to run: 20200112 亞太電信 ===
=== Now is about to run: 20200112 台固媒體 ===
=== Now is about to run: 20200112 台灣固網 ===
=== Now is about to run: 20200112 台灣大哥大 ===
=== Now is about to run: 20200112 中華電信 ===
=== Now is about to run: 20200112 凱擘 ===
=== Now is about to run: 20200112 台灣之星 ===
=== Now is about to run: 20200112 台灣基礎開發 ===
=== Now is about to run: 20200112 台灣碩網 ===


100%|██████████| 7/7 [24:12:55<00:00, 12453.70s/it]  


,tcp_dstport,ip_dst,session_i_tt_packet,frame_i_max_protocols
20200106_中嘉寬頻_China,"[7001.0, 7001.0, 7001.0, 1433.0, 1433.0, 1433....","[203.187.87.198, 203.187.87.71, 203.187.87.91,...","[1, 1, 1, 6, 4, 8, 7, 7, 3, 6, 7, 3, 4, 6, 7, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
20200106_中嘉寬頻_Ireland,"[22.0, 22.0, 22.0, 22.0, 22.0, 22.0, 22.0, 22....","[203.187.87.32, 203.187.87.32, 203.187.87.192,...","[16, 15, 14, 14, 14, 14, 14, 31, 14, 14, 14, 1...","[eth:ethertype:ip:tcp:ssh, eth:ethertype:ip:tc..."
20200106_中嘉寬頻_Taiwan,"[23.0, 23.0, 23.0, 445.0, 445.0, 445.0, 445.0,...","[203.187.87.239, 203.187.87.211, 203.187.87.22...","[1, 1, 1, 4, 2, 10, 56, 2, 4, 1, 7, 7, 7, 7, 1...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
20200106_亞太電信_China,"[1433.0, 23.0, 23.0, 445.0, 445.0, 445.0, 445....","[210.202.233.67, 222.156.247.172, 222.156.247....","[3, 1, 2, 2, 9, 1, 1, 4, 7, 1, 2, 1, 8, 4, 4, ...","[eth:ethertype:ip:tcp:tds, eth:ethertype:ip:tc..."
20200106_亞太電信_Ireland,"[3389.0, 22.0, 22.0, 22.0, 22.0, 22.0, 22.0, 2...","[210.200.17.27, 210.200.17.220, 210.200.17.236...","[6, 6, 6, 14, 6, 0, 5, 4, 0, 21, 16, 11, 11, 0...","[eth:ethertype:ip:tcp:data, eth:ethertype:ip:t..."
...,...,...,...,...
20200112_台灣基礎開發_Ireland,"[80.0, nan, 22.0, 22.0, 22.0, 22.0, 22.0, 22.0...","[61.67.24.139, 61.67.24.32, 61.67.24.192, 61.6...","[23, 1, 24, 16, 15, 12, 12, 11, 19, 12, 15, 12...","[eth:ethertype:ip:tcp:http:data, eth:ethertype..."
20200112_台灣基礎開發_Taiwan,"[445.0, 445.0, 445.0, 445.0, 445.0, 23.0, 445....","[61.67.24.128, 61.67.24.38, 61.67.24.55, 61.67...","[2, 6, 1, 1, 4, 1, 16, 2, 1, 4, 3, 2, 11, 12, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
20200112_台灣碩網_China,"[1433.0, 1433.0, 445.0, 445.0, 22.0, 1433.0, 1...","[61.64.230.114, 61.64.230.73, 61.62.173.34, 21...","[3, 4, 1, 3, 5, 1, 1, 2, 2, 1, 1, 1, 4, 1, 1, ...","[eth:ethertype:ip:tcp:tds, eth:ethertype:ip:tc..."
20200112_台灣碩網_Ireland,"[22.0, 22.0, 22.0, 22.0, 22.0, 22.0, 22.0, 22....","[61.62.66.195, 61.62.66.212, 61.62.66.252, 61....","[11, 19, 22, 11, 17, 7, 11, 12, 6, 22, 16, 13,...","[eth:ethertype:ip:tcp:ssh, eth:ethertype:ip:tc..."


In [6]:
for time in tqdm(date_li):
    for isp in isp_li:
#         if time+'_'+isp in processed_li_rev:
#             continue #之前有一部分已經跑過了        
        print("=== Now is about to run:",time,isp,"===")
        in_file = 'hdfs://192.168.50.123:8020/user/hdfs/parquet/'+str(time[:4])+'_'+str(time[4:6])+'_'+str(time[6:])+'/'+isp+'/session_parquet/'+str(time)+'_session.parquet'
#         in_file = 'hdfs://192.168.50.200/user/hdfs/parquet/'+str(time[:4])+'_'+str(time[4:6])+'_'+str(time[6:])+'/'+isp+'/session_parquet/'+str(time)+'_session.parquet'
        kdf = ks.read_parquet(in_file)
        break
    break
kdf

  0%|          | 0/7 [00:00<?, ?it/s]

=== Now is about to run: 20200106 中嘉寬頻 ===


  0%|          | 0/7 [00:11<?, ?it/s]


,session_time,session_time_list,session_duration,session_tt_packet,session_i_tt_packet,session_o_tt_packet,session_tt_frame_length,session_i_tt_frame_length,session_o_tt_frame_length,udp_tt_length,udp_i_tt_length,udp_o_tt_length,udp_i_avg_length,udp_o_avg_length,icmp_tt_length,icmp_i_tt_length,icmp_o_tt_length,icmp_i_avg_length,icmp_o_avg_length,icmp_tt_original_datagram_length,icmp_i_tt_datagram_length,icmp_o_tt_datagram_length,icmp_i_avg_datagram_length,icmp_o_avg_datagram_length,tcp_hdr_len_sum,tcp_tt_payload_length,tcp_i_tt_payload_length,tcp_o_tt_payload_length,tcp_i_avg_payload_length,tcp_o_avg_payload_length,ip_src,ip_dst,ip_proto,tcp_srcport,tcp_dstport,udp_srcport,udp_dstport,country,domain,isp,frame_i_common_protocols,frame_o_common_protocols,frame_i_max_protocols,frame_o_max_protocols,tcp_i_payload_list,tcp_o_payload_list
0,1.578289e+09,"[1578289239.01777, 1578289242.020537, 15782892...",9.002928,3,3,0,194,194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92,0,0,NaN,0.000000,NaN,1.0.150.241,203.187.87.27,6,54135,445,NaN,NaN,Thailand,totinternet.net,None,eth:ethertype:ip:tcp,None,eth:ethertype:ip:tcp,None,[],[]
1,1.578293e+09,"[1578293174.029128, 1578293174.029226, 1578293...",1.008835,19,8,11,1905,899,1006.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,404,819,455,364.0,56.875000,33.090909,1.0.150.241,203.187.87.99,6,54371,445,NaN,NaN,Thailand,totinternet.net,None,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,eth:ethertype:ip:tcp:nbss:smb,[[6:wV08dY0TAqRARUbESX1qvasTgb+RpoedfFKXqEPdcV...,[[6:iCV00dYMe9hNV9oC81sbdoa0VfCy/a9evgfey9FV1R...
2,1.578309e+09,"[1578308930.021367, 1578308930.021958, 1578308...",38.305965,43,27,16,32000,16656,15344.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1272,29266,14986,14280.0,555.037037,892.500000,1.10.188.52,203.187.87.131,6,51582,80,NaN,NaN,Thailand,totinternet.net,None,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:http:urlencoded-form,eth:ethertype:ip:tcp:http:data-text-lines,[[24:uAawYD5K/S18YH+w18YHxwdN04adN/QDdlUX18YHZ...,[[96:DVb4NF1Cp4QuQxr9sHjqozJnsnSnrnb01gH1FUyYA...
3,1.578313e+09,"[1578312789.421785, 1578312789.421954, 1578312...",0.925712,19,8,11,1905,899,1006.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,404,819,455,364.0,56.875000,33.090909,1.10.203.117,203.187.87.103,6,27211,445,NaN,NaN,Thailand,totinternet.net,None,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,eth:ethertype:ip:tcp:nbss:smb,[[6:wV08dY0TAqRARUbESX1qvasTgb+RpoedfFKXqEPdcV...,[[6:iCV00dYMe9hNV9oGgoa0VfCy/a9evgfey9FV1RG/5r...
4,1.578304e+09,"[1578304147.732373, 1578304147.732523, 1578304...",0.811230,18,8,10,1845,899,946.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,384,819,455,364.0,56.875000,36.400000,1.10.203.117,203.187.87.72,6,29075,445,NaN,NaN,Thailand,totinternet.net,None,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,eth:ethertype:ip:tcp:nbss:smb,[[6:wV08dY0TAqRARUbESX1qvasTgb+RpoedfFKXqEPdcV...,[[6:iCV00dYMe9hNV9oY6oa0VfCy/a9evgfey9FV1RG/5r...
5,1.578303e+09,"[1578303253.637298, 1578303253.637538]",0.000240,2,1,1,118,58,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44,0,0,0.0,0.000000,0.000000,1.119.161.187,203.187.87.198,6,43275,7001,NaN,NaN,China,None,None,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,[],[]
6,1.578303e+09,"[1578303322.166098, 1578303322.166241]",0.000143,2,1,1,118,58,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44,0,0,0.0,0.000000,0.000000,1.119.161.187,203.187.87.71,6,43275,7001,NaN,NaN,China,None,None,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,[],[]
7,1.578303e+09,"[1578303290.553051, 1578303290.553315]",0.000264,2,1,1,118,58,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44,0,0,0.0,0.000000,0.000000,1.119.161.187,203.187.87.91,6,43275,7001,NaN,NaN,China,None,None,eth:et

In [7]:
kdf.columns

Index(['session_time', 'session_time_list', 'session_duration',
       'session_tt_packet', 'session_i_tt_packet', 'session_o_tt_packet',
       'session_tt_frame_length', 'session_i_tt_frame_length',
       'session_o_tt_frame_length', 'udp_tt_length', 'udp_i_tt_length',
       'udp_o_tt_length', 'udp_i_avg_length', 'udp_o_avg_length',
       'icmp_tt_length', 'icmp_i_tt_length', 'icmp_o_tt_length',
       'icmp_i_avg_length', 'icmp_o_avg_length',
       'icmp_tt_original_datagram_length', 'icmp_i_tt_datagram_length',
       'icmp_o_tt_datagram_length', 'icmp_i_avg_datagram_length',
       'icmp_o_avg_datagram_length', 'tcp_hdr_len_sum',
       'tcp_tt_payload_length', 'tcp_i_tt_payload_length',
       'tcp_o_tt_payload_length', 'tcp_i_avg_payload_length',
       'tcp_o_avg_payload_length', 'ip_src', 'ip_dst', 'ip_proto',
       'tcp_srcport', 'tcp_dstport', 'udp_srcport', 'udp_dstport', 'country',
       'domain', 'isp', 'frame_i_common_protocols', 'frame_o_common_protocols',
       

In [2]:
date_li = ['20200106','20200107','20200108','20200109','20200110',
           '20200111','20200112'] #改
# date_li.reverse()
isp_li = ['中嘉寬頻','亞太電信','台固媒體','台灣固網','台灣大哥大','中華電信','凱擘','台灣之星',
        '台灣基礎開發','台灣碩網']
# protocols_need = ['http','mysql','ftp','smb','smtp','imap','pop','rpc','ssh','telnet','sip']

# isp_li.reverse()
file_name = "_".join(sorted(date_li)) #date_li 改!!
min_date = str(min(date_li))
max_date = str(max(date_li))


In [3]:
# tri_country_df = pd.DataFrame()
# port_dist_df = pd.DataFrame()
tri_country_df = pickle.load(open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/IrelandCnTw_beforepreprocess_dist.pkl','rb'))
tri_country_df

,tcp_dstport,session_i_tt_packet,tcp_i_tt_payload_length
20200106_中嘉寬頻_China,"[1433, 1433, 1433, 1433, 1433, 1433, 1433, 143...","[6, 8, 6, 7, 6, 7, 8, 8, 7, 8, 8, 7, 6, 6, 6, ...","[300, 398, 347, 400, 351, 349, 351, 398, 408, ..."
20200106_中嘉寬頻_Ireland,"[22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 2...","[16, 15, 14, 14, 14, 14, 14, 31, 14, 14, 14, 1...","[713, 713, 1312, 1312, 1312, 1312, 1312, 3264,..."
20200106_中嘉寬頻_Taiwan,"[445, 21, 21, 21, 21, 22, 22, 2222, 2222, 2222...","[56, 7, 7, 7, 7, 14, 14, 14, 16, 17, 16, 15, 1...","[63938, 33, 25, 31, 29, 886, 886, 886, 886, 88..."
20200106_亞太電信_China,"[22, 80, 80, 80, 80, 8080, 445, 1433, 1433, 14...","[8, 9, 10, 16, 20, 6, 7, 7, 6, 6, 7, 10, 5, 6,...","[541, 531, 758, 1249, 1523, 190, 191, 384, 190..."
20200106_亞太電信_Ireland,"[22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 2...","[6, 6, 6, 5, 11, 11, 13, 23, 15, 20, 54, 16, 1...","[12, 12, 12, 12, 1092, 956, 956, 960, 1412, 13..."
20200106_亞太電信_Taiwan,"[1433, 445, 445, 1433, 1433, 1433, 1433, 1433,...","[10, 8, 6, 8, 8, 13, 6, 10, 11, 10, 6, 6, 6, 9...","[575, 455, 267, 388, 378, 570, 184, 426, 582, ..."
20200106_台固媒體_China,"[445, 80, 80, 445, 22, 445, 80, 80, 80, 80, 14...","[8, 128, 117, 7, 14, 6, 6, 6, 6, 6, 6, 6, 7, 5...","[364, 302, 302, 442, 913, 364, 267, 271, 269, ..."
20200106_台固媒體_Ireland,"[22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 2...","[14, 14, 14, 14, 14, 14, 47, 55, 14, 14, 14, 2...","[1312, 1312, 1312, 1312, 1312, 1312, 3844, 616..."
20200106_台固媒體_Taiwan,"[445, 445, 445, 445, 445, 1433, 1433, 1433, 14...","[18, 63, 8, 6, 8, 7, 8, 8, 5, 8, 6, 6, 6, 5, 6...","[980, 64349, 455, 364, 455, 192, 156, 384, 190..."
20200106_台灣固網_China,"[80, 445, 80, 21, 21, 21, 21, 21, 21, 1433, 14...","[16, 6, 6, 27, 27, 27, 27, 27, 23, 11, 8, 13, ...","[14444, 364, 304, 225, 222, 225, 221, 226, 139..."


In [ ]:

# take_columns = ['session_i_tt_packet','session_o_tt_packet']
# pkt_sess_df = pd.DataFrame(columns=isp_li,index=date_li)
# country_count_dict = {}
# protocol_count_dict = {}

# counter = 0
for da in tqdm(date_li):
    for isp in isp_li:
        if da+'_'+isp in processed_li_rev:
            continue #之前有一部分已經跑過了
        print("=== Now is about to run:",da,isp,"===")
        
        dir_path = "/mnt/Raid160TB/pcap_inter/"+str(da[:4])+'_'+str(da[4:6])+'_'+str(da[6:])+"/"+isp+"/intermeidate_data/"
        saved_path = dir_path+"df2_tuples_"+isp+".pkl"
        gc.collect()
        df2,pickle_dir,picture_dir = pickle.load(open(saved_path,'rb')) 
        
        temp1_master = df2[['country','tcp_dstport','session_i_tt_packet','tcp_i_tt_payload_length']]
        temp1 = temp1_master.groupby(['country'])
        temp1 = temp1.aggregate(list)
        temp11 = temp1[temp1.index.isin(['China','Taiwan','Ireland'])] #指定需要的國家
        t_index = temp11.index.tolist()
        t_index = [da+'_'+isp+'_'+ind for ind in t_index]
        temp11.index = t_index
        tri_country_df = tri_country_df.append(temp11)
        
#         temp1_master = df2[['country','tcp_dstport','session_i_tt_packet','session_o_tt_packet','tcp_i_tt_payload_length','tcp_o_tt_payload_length','frame_i_max_protocols','frame_o_max_protocols','tcp_i_payload_list','tcp_o_payload_list']]
#         temp2 = temp1_master.groupby(['tcp_dstport'])
#         temp2 = temp2.aggregate(list)
#         t_index = temp2.index.tolist()
#         t_index = [da+'_'+isp+'_'+str(ind) for ind in t_index]
#         temp2.index = t_index
#         port_dist_df = port_dist_df.append(temp2)
        gc.collect()
        #checkpoint
        pickle.dump(obj=tri_country_df,file=open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/IrelandCnTw_beforepreprocess_dist.pkl','wb'))
#         pickle.dump(obj=port_dist_df,file=open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/all_port_beforepreprocess_dist.pkl','wb'))
#         break
#     break
# print(df2.columns) #tcp_dstport、session_duration、session_i_tt_packet、tcp_i_tt_payload_length、
# df2
tri_country_df

  0%|          | 0/7 [00:00<?, ?it/s]

=== Now is about to run: 20200106 台灣之星 ===
=== Now is about to run: 20200106 台灣基礎開發 ===
=== Now is about to run: 20200106 台灣碩網 ===


 14%|█▍        | 1/7 [02:30<15:01, 150.22s/it]

=== Now is about to run: 20200107 中嘉寬頻 ===
=== Now is about to run: 20200107 亞太電信 ===
=== Now is about to run: 20200107 台固媒體 ===
=== Now is about to run: 20200107 台灣固網 ===
=== Now is about to run: 20200107 台灣大哥大 ===
=== Now is about to run: 20200107 中華電信 ===
=== Now is about to run: 20200107 凱擘 ===
=== Now is about to run: 20200107 台灣之星 ===
=== Now is about to run: 20200107 台灣基礎開發 ===
=== Now is about to run: 20200107 台灣碩網 ===


 29%|██▊       | 2/7 [3:40:12<10:45:27, 7745.54s/it]

...

=== Now is about to run: 20200109 中嘉寬頻 ===
=== Now is about to run: 20200109 亞太電信 ===
=== Now is about to run: 20200109 台固媒體 ===
=== Now is about to run: 20200109 台灣固網 ===
=== Now is about to run: 20200109 台灣大哥大 ===
=== Now is about to run: 20200109 中華電信 ===
=== Now is about to run: 20200109 凱擘 ===
=== Now is about to run: 20200109 台灣之星 ===
=== Now is about to run: 20200109 台灣基礎開發 ===
=== Now is about to run: 20200109 台灣碩網 ===


 57%|█████▋    | 4/7 [7:17:46<5:22:08, 6442.99s/it] 

=== Now is about to run: 20200110 中嘉寬頻 ===
=== Now is about to run: 20200110 亞太電信 ===
=== Now is about to run: 20200110 台固媒體 ===
=== Now is about to run: 20200110 台灣固網 ===
=== Now is about to run: 20200110 台灣大哥大 ===
=== Now is about to run: 20200110 中華電信 ===
=== Now is about to run: 20200110 凱擘 ===
=== Now is about to run: 20200110 台灣之星 ===
=== Now is about to run: 20200110 台灣基礎開發 ===
=== Now is about to run: 20200110 台灣碩網 ===


 71%|███████▏  | 5/7 [11:39:27<5:26:03, 9781.68s/it]

=== Now is about to run: 20200111 中嘉寬頻 ===
=== Now is about to run: 20200111 亞太電信 ===
=== Now is about to run: 20200111 台固媒體 ===


In [4]:
processed_li = tri_country_df.index.tolist()
processed_li_rev = ["_".join(x.split('_')[:2]) for x in processed_li]
processed_li_rev = list(set(processed_li_rev))
processed_li_rev

['20200106_台灣固網',
 '20200106_中華電信',
 '20200106_台灣大哥大',
 '20200106_凱擘',
 '20200106_亞太電信',
 '20200106_台固媒體',
 '20200106_中嘉寬頻']

In [19]:
temp1 = df2[['country','tcp_dstport','session_i_tt_packet','tcp_i_tt_payload_length']]
temp1 = temp1.groupby(['country'])
temp1 = temp1.aggregate(list)
temp1
# df2[df2.country == 'China'] #Ireland、China、Taiwan

,tcp_dstport,session_i_tt_packet,tcp_i_tt_payload_length
country,,,
Afghanistan,[80],[177],[185]
Albania,"[8080, 80, 80, 80, 8080, 80, 8080, 80, 80, 80,...","[6, 7, 21, 11, 6, 69, 6, 7, 6, 22, 24, 242, 36...","[185, 186, 13909, 192, 192, 14533, 187, 185, 1..."
Algeria,"[445, 445, 445, 445, 445, 445, 445, 445, 445, ...","[6, 6, 6, 6, 8, 8, 13, 54, 8, 6, 6, 8, 6, 8, 8...","[364, 364, 364, 364, 455, 455, 866, 63938, 455..."
Angola,[22],[18],[1175]
Antigua and Barbuda,"[22, 22, 22]","[17, 17, 18]","[1175, 1175, 1079]"
...,...,...,...
Vietnam,"[445, 2222, 2222, 445, 2222, 2222, 22, 22, 22,...","[12, 13, 13, 6, 13, 13, 13, 14, 8, 14, 40, 10,...","[592, 713, 713, 364, 713, 713, 729, 932, 651, ..."
Yemen,"[445, 445, 445, 445, 445, 445, 445, 445, 445, ...","[4, 8, 8, 6, 8, 8, 8, 8, 8, 8, 13, 8, 18, 8, 6...","[191, 364, 455, 364, 455, 455, 442, 364, 455, ..."
Zambia,"[22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 2...","[13, 13, 13, 16, 12, 13, 12, 13, 12, 15, 13, 1...","[713, 713, 713, 713, 745, 713, 745, 713, 745, ..."


In [28]:
temp11 = temp1[temp1.index.isin(['China','Taiwan','Ireland'])]
t_index = temp11.index.tolist()
t_index = [da+'_'+isp+'_'+ind for ind in t_index]
temp11.index = t_index
temp11

,tcp_dstport,session_i_tt_packet,tcp_i_tt_payload_length
20200106_中嘉寬頻China,"[1433, 1433, 1433, 1433, 1433, 1433, 1433, 143...","[6, 8, 6, 7, 6, 7, 8, 8, 7, 8, 8, 7, 6, 6, 6, ...","[300, 398, 347, 400, 351, 349, 351, 398, 408, ..."
20200106_中嘉寬頻Ireland,"[22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 2...","[16, 15, 14, 14, 14, 14, 14, 31, 14, 14, 14, 1...","[713, 713, 1312, 1312, 1312, 1312, 1312, 3264,..."
20200106_中嘉寬頻Taiwan,"[445, 21, 21, 21, 21, 22, 22, 2222, 2222, 2222...","[56, 7, 7, 7, 7, 14, 14, 14, 16, 17, 16, 15, 1...","[63938, 33, 25, 31, 29, 886, 886, 886, 886, 88..."


In [46]:
temp1_master = df2[['country','tcp_dstport','session_i_tt_packet','session_o_tt_packet','tcp_i_tt_payload_length','tcp_o_tt_payload_length','frame_i_max_protocols','frame_o_max_protocols','tcp_i_payload_list','tcp_o_payload_list']]
temp2 = temp1_master.groupby(['tcp_dstport'])
temp2 = temp2.aggregate(list)
t_index = temp2.index.tolist()
t_index = [da+'_'+isp+'_'+str(ind) for ind in t_index]
temp2.index = t_index
temp2

,country,session_i_tt_packet,session_o_tt_packet,tcp_i_tt_payload_length,tcp_o_tt_payload_length,frame_i_max_protocols,frame_o_max_protocols,tcp_i_payload_list,tcp_o_payload_list
20200106_中嘉寬頻_21,"[United States, Taiwan, Taiwan, Taiwan, Taiwan...","[6, 7, 7, 7, 7, 7, 7, 7, 8, 11, 11, 11, 5, 8, ...","[6, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10...","[10, 33, 25, 31, 29, 24, 33, 28, 41, 34, 37, 3...","[51, 113, 113, 113, 113, 113, 113, 113, 182, 1...","[eth:ethertype:ip:tcp:ftp, eth:ethertype:ip:tc...","[eth:ethertype:ip:tcp:ftp, eth:ethertype:ip:tc...","[[[3:2f0/9B9En:2f0/j9En, 1578249914.441039000,...",[[[3:QCWVSsTQCfyNTQ/OP/1Tpv9duESXjm:QCSSsTbJmd...
20200106_中嘉寬頻_22,"[Thailand, South Korea, United States, China, ...","[13, 13, 13, 13, 13, 14, 14, 10, 13, 14, 14, 1...","[15, 15, 15, 15, 15, 15, 15, 16, 15, 15, 19, 1...","[729, 713, 713, 713, 729, 886, 729, 509, 713, ...","[2092, 2092, 2092, 2092, 2092, 2044, 2092, 486...","[eth:ethertype:ip:tcp:ssh, eth:ethertype:ip:tc...","[eth:ethertype:ip:tcp:ssh, eth:ethertype:ip:tc...","[[[3:DfQMPmfK7PATaupm:DPQKtupm, 1578254105.213...",[[[3:DfQMPmflfTAfQW4Y7wUZIr6drA60vA6Eia1rm:DPQ...
20200106_中嘉寬頻_23,"[Romania, Romania, Romania, Romania, Romania, ...","[6, 6, 6, 6, 13, 6, 6, 6, 13, 6, 10, 6, 6, 5, ...","[5, 5, 5, 5, 7, 5, 5, 5, 6, 4, 6, 5, 4, 4, 5, ...","[33, 33, 4, 4, 24, 4, 4, 4, 954, 8, 16, 33, 48...","[27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 2...","[eth:ethertype:ip:tcp:telnet, eth:ethertype:ip...","[eth:ethertype:ip:tcp:telnet, eth:ethertype:ip...",[[[3:3D/UdfrzC5SW/J2DrV7/UCVfFfrfVJDd/:gdCf+ra...,"[[[3:UNT2CIQ7A4fTxx18uZhEn:UgNIrfdxSiEn, 15782..."
20200106_中嘉寬頻_25,"[Vietnam, China, Sweden, Poland, Poland, Polan...","[40, 7, 6, 6, 10, 12, 8, 5, 6, 6, 7, 8, 7, 7, ...","[22, 8, 5, 8, 14, 16, 12, 6, 8, 8, 10, 11, 10,...","[32178, 15, 27, 31, 35, 35, 35, 11, 32, 32, 29...","[135, 54, 40, 54, 64, 64, 64, 38, 54, 54, 56, ...",[eth:ethertype:ip:tcp:smtp:imf:data-text-lines...,"[eth:ethertype:ip:tcp:smtp, eth:ethertype:ip:t...",[[[3:RlhfRcfR6P+5EfRGYdfqIW+aBfu:RlJRe6Pd7qFpB...,"[[[3:QCWVSsTQCfyNTQ97E:QCSSsTbJpE, 1578325360...."
20200106_中嘉寬頻_42,"[Russia, Russia, Russia, Russia, Russia, Germa...","[8, 8, 8, 8, 8, 8, 5, 8, 8, 8, 5, 5, 8, 8, 8, ...","[8, 7, 8, 8, 8, 7, 4, 7, 7, 7, 4, 5, 8, 8, 8, ...","[465, 465, 465, 465, 465, 465, 47, 465, 465, 4...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...",[[[3:OXXXL1Vf1V1V1V1RYP/LfIQfdplr+M5DcEYVV91v/...,"[[[3:Sn:Sn, 1578284678.743698000, 2]], [[3:Sn:..."
20200106_中嘉寬頻_80,"[Thailand, China, China, India, Hong Kong, Ind...","[27, 38, 3473, 7, 6, 35, 5, 36, 122, 7, 6, 10,...","[16, 42, 2521, 6, 5, 29, 5, 22, 80, 6, 5, 5, 4...","[14986, 18596, 15544, 185, 256, 14419, 232, 14...","[14280, 48580, 17593647, 96, 446, 85432, 446, ...","[eth:ethertype:ip:tcp:http:urlencoded-form, et...","[eth:ethertype:ip:tcp:http:data-text-lines, et...",[[[24:uAawYD5K/S18YH+w18YHxwdN04adN/QDdlUX18YH...,[[[96:DVb4NF1Cp4QuQxr9sHjqozJnsnSnrnb01gH1FUyY...
20200106_中嘉寬頻_110,"[United States, United States, Romania, Nether...","[7, 7, 7, 7, 8, 7, 9, 7, 7, 7, 7, 5, 8, 12, 7,...","[6, 6, 6, 6, 6, 6, 10, 6, 6, 6, 6, 5, 7, 11, 6...","[10, 6, 18, 204, 18, 18, 71, 18, 18, 18, 18, 6...","[18, 18, 18, 18, 18, 18, 164, 18, 18, 18, 18, ...","[eth:ethertype:ip:tcp:pop, eth:ethertype:ip:tc...","[eth:ethertype:ip:tcp:pop:imf, eth:ethertype:i...","[[[3:2f0/9B9En:2f0/j9En, 1578240246.977626000,...","[[[3:QCWVRBf5jWfPSXjm:QCScfqa, 1578240246.8144..."
20200106_中嘉寬頻_111,"[Vietnam, Vietnam, Vietnam, Vietnam, Vietnam, ...","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[120, 120, 120, 120, 120, 120, 120, 120, 120, ...","[64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 6...","[eth:ethertype:ip:tcp:rpc:portmap, eth:etherty...","[eth:ethertype:ip:tcp:rpc:portmap, eth:etherty...",[[[3:PVVfXWdu469V1V1V1V1V1VP/B1XV1VP/vV1VYvV1V...,[[[3:PVVf

In [26]:
Counter(temp1.loc['China']['tcp_dstport'])

Counter({1433: 69164,
         2222: 39694,
         80: 19158,
         22: 158906,
         445: 4017,
         3389: 1733,
         8080: 297,
         25: 131,
         21: 480,
         139: 2388,
         3306: 899,
         1023: 113,
         9999: 18,
         135: 70,
         443: 60,
         1025: 112,
         2954: 61,
         587: 14,
         3128: 16,
         1080: 5,
         143: 3,
         23: 2})

In [ ]:
dir_path = "/mnt/Raid160TB/pcap_inter/"+str(date[:4])+'_'+str(date[4:6])+'_'+str(date[6:])+"/"+isp+"/intermeidate_data/"
df2,pickle_dir,picture_dir = pickle.load(open(dir_path+"df2_tuples_"+isp+".pkl",'rb'))
df2